In [2]:
from fiftyone import ViewField as F
import fiftyone as fo
import numpy as np
from tqdm import tqdm

dataset = fo.load_dataset("mcity_fisheye_3_months")

In [4]:
try:
    dataset.delete_sample_field("bbox_area_abs")
    dataset.delete_sample_field("bbox_area_rel")
except:
    pass

In [22]:
bbox_areas = []
pred_fields = []

#Get fields of zero shot model predictions
dataset_schema = dataset.get_field_schema()
for field in dataset_schema:
    if "pred_" in field:
        pred_fields.append(field)

# Filter data, iterate over detections, and compute bounding box areas
for field in pred_fields:
    print(field)
    view = (
        dataset
        .exists(field)
        .match(F(f"{field}.detections") != [])
    )
    samples_detections = view.values(field + ".detections") # list of lists, one list of detections per sample
    for detections in tqdm(samples_detections):
        for detection in detections:
            bbox_area = float(detection["bounding_box"][2] * detection["bounding_box"][3])
            if (isinstance(bbox_area, (int, float)) and 0 <= bbox_area <= 1) == False:
                print(bbox_area)
            detection["bbox_area"] = bbox_area
            bbox_areas.append(bbox_area)

    # Save the updated detections
    view.set_values(field + ".detections", samples_detections)    

print(len(bbox_areas))

pred_google_owlvit_large_patch14


100%|██████████| 30434/30434 [00:00<00:00, 72583.11it/s]


pred_IDEA_Research_grounding_dino_base


100%|██████████| 2204/2204 [00:00<00:00, 96240.09it/s]


pred_google_owlv2_base_patch16_ensemble


...

pred_google_owlv2_large_patch14_ensemble


100%|██████████| 35510/35510 [00:00<00:00, 67929.05it/s]


pred_omlab_omdet_turbo_swin_tiny_hf


100%|██████████| 49312/49312 [00:11<00:00, 4402.95it/s]


1272493


In [ ]:
dataset = dataset.take(100)
for sample in tqdm(dataset):
    #print(sample)
    for detections in sample[pred_fields[0]].detections:
        for detection in detections:
            print(detection))

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: string indices must be integers, not 'str'

In [20]:
bbox_areas = []

field = pred_fields[0]
print(field)
view = (
    dataset
    .exists(field)
    .match(F(f"{field}.detections") != [])
)
samples_detections = view.values(field + ".detections") # list of lists, one list of detections per sample
for detections in tqdm(samples_detections):
    for detection in detections:
        #print(detection)
        bbox_areas.append(detection["bbox_area"])

for area in bbox_areas:
    if area < 0 or area > 1:
        print(area)

print(len(bbox_areas))

pred_google_owlvit_large_patch14


100%|██████████| 30434/30434 [00:00<00:00, 1417618.39it/s]

49718


In [14]:
# Compute bins
num_bins = 10
quantiles = np.linspace(0, 1, num_bins + 1)
quantile_bin_edges = np.quantile(bbox_areas, quantiles)
upper_bin_limits = list(quantile_bin_edges[1:-1]) + [np.float64(1)]
print(upper_bin_limits)

[np.float64(0.0002966239083589244), np.float64(0.00046908459728911117), np.float64(0.0006772121438674127), np.float64(0.001025925307549187), np.float64(0.0014855543452785014), np.float64(0.0020463086280597055), np.float64(0.0033701701579169218), np.float64(0.009457432087225263), np.float64(0.8137730338420947), np.float64(1.0)]


In [15]:
# Compute bins
num_bins = 10
quantiles = np.linspace(0, 1, num_bins + 1)
quantile_bin_edges = np.quantile(bbox_areas, quantiles)
upper_bin_limits = list(quantile_bin_edges[1:-1]) + [np.float64(1)]
print(upper_bin_limits)

[np.float64(0.0002966239083589244), np.float64(0.00046908459728911117), np.float64(0.0006772121438674127), np.float64(0.001025925307549187), np.float64(0.0014855543452785014), np.float64(0.0020463086280597055), np.float64(0.0033701701579169218), np.float64(0.009457432087225263), np.float64(0.8137730338420947), np.float64(1.0)]


In [ ]:
# Assign bin label tags to detections
for field in pred_fields:
    print(field)
    view = (
        dataset
        .exists(field)
        .match(F(f"{field}.detections") != [])
    )
    samples_detections = view.values(field + ".detections") # list of lists, one list of detections per sample
    for detections in tqdm(samples_detections):
        for detection in detections:
            bin_index = np.digitize(
                    detection["bbox_area"], upper_bin_limits, right=True
                )
            bin_upper_limit = "{:.4f}".format(upper_bin_limits[bin_index])
            bin_name = "bbox_size_bin_" + str(bin_index) + "_" + bin_upper_limit
            detection.tags.append(bin_name)

    # Save the assigned bin tag
    view.set_values(field + ".detections", samples_detections)    

pred_google_owlvit_large_patch14


100%|██████████| 30434/30434 [00:00<00:00, 89615.03it/s]


pred_IDEA_Research_grounding_dino_base


100%|██████████| 2204/2204 [00:00<00:00, 134536.12it/s]


pred_google_owlv2_base_patch16_ensemble


100%|██████████| 26753/26753 [00:00<00:00, 71172.59it/s]


pred_google_owlv2_large_patch14_ensemble


100%|██████████| 35510/35510 [00:00<00:00, 91723.52it/s]


pred_omlab_omdet_turbo_swin_tiny_hf


100%|██████████| 49312/49312 [00:08<00:00, 5998.02it/s] 


In [ ]:
#for sample in dataset.iter_samples(progress=True, autosave=True, batch_size=1000):
#    for field in sample.iter_fields():
#        if "pred_" in field[0]:
#            for detection in field[1]["detections"]:
#                bin_index = np.digitize(
#                    detection["bbox_area"], upper_bin_limits, right=True
#                )
#                bin_upper_limit = "{:.4f}".format(upper_bin_limits[bin_index])
#                bin_name = "bin_" + str(bin_index) + "_" + bin_upper_limit
#                detection["bbox_area_bin"] = bin_name